# CANDO Tutorial

This notebook will walk you through how to generate a CANDO matrix, set up a CANDO object, probe the data, benchmark the platform, and make therapeutic predictions. 

## Introduction

## Getting Started

We begin by importing the cando package. Once cando has been imported we can pull the example mappings and matrix data for this tutorial, `cnd.get_tutorial()`. Lastly, for our set up, we define important variables that will be used throughout this tutorial. These variables will be explained as each becomes important for different functions.

In [1]:
import sys, os
# import the cando package
import cando as cnd

# Pull data for this entire tutorial
cnd.get_tutorial()
cnd.get_v2()

# Define variables for matrix generation and CANDO object creation
matrix_file='example-matrix.tsv'
cmpd_map='../v2.0/mappings/drugbank-approved.tsv'
ind_map='../v2.0/mappings/ctd_2_drugbank.tsv'
cmpd_scores='../v2.0/cmpds/scores/drugbank-approved-rd_ecfp4.tsv' # ~400MB, may take a while
prot_scores='example-prots_scores.tsv'
protein_set="example-bac-prots.txt"
dist_metric='rmsd'
ncpus=3

os.chdir("./examples")

All data for tutorial downloaded.
All data for v2.0 downloaded.


## Generating interaction matrix

<span style="color:red">**This step is optional**</span> - The example interaction matrix, `matrix_file`, has already been downloaded via `get_tutorial()`. This function may take a few minutes.

In this step we will generate a matrix of 2,162 drugs by 64 proteins, populated with the corresponding interaction score betwen each drug and protein. The final matrix will have drugs as the columns (indexed according to the compound mapping file), and proteins as the indices (indexed by PDB and chain ID).

The function `generate_matrix()` first creates a dataframe for all tanimoto scores comparing each drug in our dataset (2,162) to every potential binding site ligand from the PDB, `cmpd_scores`. Next, it creates a dataframe of all potential binding sites for each protein in our dataset (example set = 64 proteins) with the corresponding binding site score, `prot_scores`. The function then iterates over all drugs and protein binding sites to populate the matrix with the best score based on the following input parameters: 1) `interaction_score` - the scoring protocol of choice, which can be 'C', 'dC', 'P', 'CxP', 'dCxP', where C is the Tanimoto score, dC is the percentile of the Tanimoto score for the compound, and P is binding site score from the [COACH algorithm](https://zhanglab.ccmb.med.umich.edu/COACH/), 2) `c_cutoff` and `p_cutoff` - set cutoffs which ignore any Tanimoto or binding site score below each threshold, respectively, and 3) `percentile_cutoff` - similar to `c_cutoff` but with the percentile of the Tanimoto score (overrides `c_cutoff` if not `None`). These scores represent how strongly each drug may potentially bind to each protein target. We then output the matrix to a tsv file, `matrix_file`.

We will create a matrix using the 'C' protocol, which just chooses the top Tanimoto score to any ligand predicted to bind to a given protein, without any cutoffs. This function is parallelized, so setting the vairable `ncpus` will change the number of processors that are used for this function. NOTE: percentile cutoff protocols ('dC', 'dCxP') take much longer to compute than 'C' and 'P' protocols. 

In [2]:
# generate example cando interaction matrix (2,162 drugs x 64 proteins)
cnd.generate_matrix(matrix_file=matrix_file, cmpd_scores=cmpd_scores,
                    prot_scores=prot_scores, interaction_score='C', ncpus=ncpus)

Compiling compound scores...
Compiling binding site scores...
Calculating interaction scores...
Generating matrix...
Matrix written to example-matrix.tsv.
Matrix generation took 4 min 0 s to finish.


## Setting up CANDO object

1. First argument, `cmpd_map`, is the compound mapping which specifies all of the compounds in the matrix by name and ID.

2. Second argument, `ind_map`, is the indication mapping which specifies which diseases the drugs/compounds are approved to treat.

3. `matrix` is the cando interaction matrix file which contains the names of the proteins and the scores to all compounds. **This file must be in tsv format.**

4. `compute_distance` tells the object to compute the distances between the compounds in the platform based on the similarity of their interactions with all of the proteins. The distance metric used, `dist_metric`, can be cosine, rmsd, or many other common distance metrics (default is rmsd). This can be relatively time consuming depending on the amount of proteins, but with proteins on the order of hundreds it shouldn't take longer than a minute or two. If the computation does take a while (let's say >100,000 proteins), you can add a `save_dists='name_of_dist_file.tsv'` flag to save the computation. Then, simply use the `read_dists='name_of_dist_file.tsv'` to read in the already computed RMSDs to save time. 

In [3]:
# create object using example cando interaction matrix
# compute distances using rmsd distance metric
cando = cnd.CANDO(cmpd_map, ind_map, matrix=matrix_file, compute_distance=True, 
                  dist_metric=dist_metric, ncpus=ncpus)

Reading signatures from matrix...
Done reading signatures.

Computing rmsd distances...
Done computing rmsd distances.



### Check data
The imported data from `get_tutorial()` contains the CANDO v2.0 mappings with a sample of 64 proteins (for simplicity sake). It should contain the v2.0 set of 2,162 drugs/compounds and 2,178 indications. Let's check to make sure this is the case. The `CANDO` object should automagically assign each compound its signature of 64 proteins. 

We can also search for compounds using a string input with `search_compound()`, in case you are unaware if the mapping file contains a compound of interest. Don't worry about exact spelling, it will output the top 5 (default) closest matches to your query. 

In [4]:
# print cando object stats
print('compounds', len(cando.compounds))
print('indications', len(cando.indications))
print('proteins', len(cando.proteins))
print('')

cando.search_compound('bilavarudin')

compounds 2162
indications 2178
proteins 64

id	name
0	bivalirudin
570	lamivudine
1748	ivabradine
1102	telbivudine
1508	nilvadipine


Spelling was a bit off, but we found bivalirudin!

Let's make sure bivalirudin has 64 values in its signature and take a look at the values themselves. We know its `id_` is 0 from searching. Depending on the scoring protocol used, the range/distribution of these values will vary.

The `compute_distance` flag above tells the CANDO object to compute the RMSD between each compound on an all-vs-all basis. Let's check out the top5 most similar compounds to bivalirudin. The `Compound.similar` lists are part of the `Compound` objects and contain a tuple of every other compound object and its computed RMSD.

In [5]:
# print bivalirudin signature
c = cando.compounds[0]
print(c.name, len(c.sig))
print(c.sig)
print('')

# top5 most similar compounds to bivalirudin
for s in c.similar[0:5]:
    print(s[0].name, round(s[1], 3))

bivalirudin 64
[0.496, 0.482, 0.385, 0.061, 0.0, 0.11, 0.114, 0.375, 0.109, 0.102, 0.12, 0.418, 0.227, 0.0, 0.417, 0.146, 0.109, 0.129, 0.09, 0.099, 0.137, 0.109, 0.1, 0.109, 0.04, 0.133, 0.101, 0.19, 0.129, 0.077, 0.09, 0.571, 0.134, 0.202, 0.108, 0.408, 0.211, 0.0, 0.188, 0.432, 0.136, 0.116, 0.0, 0.0, 0.195, 0.535, 0.143, 0.232, 0.021, 0.056, 0.09, 0.102, 0.447, 0.114, 0.135, 0.01, 0.127, 0.188, 0.039, 0.096, 0.114, 0.214, 0.128, 0.405]

goserelin 0.053
gramicidin_d 0.07
desmopressin 0.088
cetrorelix 0.051
daptomycin 0.086


## Canbenchmark
An important part of CANDO is benchmarking how well we can recapture drugs known to treat the same dieases within their respective `Compound.similar` lists. Currently, our benchmarking code calculates three metrics:
1. Average indication accuracy (aia) - this value is the average of every individual indication accuracy
2. Average pairwise accuracy (apa) - this value is the weighted average of each individual indication accuracy based on the number of drugs approved to treat it
3. Indication coverage (ic) - this is the count of the number of non-zero indication accuracies

The higher the metric scores for a given matrix/compound-protein scoring protocol, the more confidence we will have in predictions made using it. 

An accuracy is calculated for each indication that has at least 2 compounds associated. To do this, we hold out each compound and look for any of the other approved compounds within a certain cutoff of the `Compound.similar` list for the held-out compound. The cutoffs are predetermined to be top10, top25, top50, and top100. There are also percent cutoffs that vary based on the number of compounds in the platform. So, for Indication-A with three drugs approved (D1, D2, and D3), we would hold out D1 and look for *either* D2 or D3 in the top10, top25, top50, and top100 compounds to D1. This would be repeated for D2 and D3. So let's say D3 was recaptured at rank 5 for D1, D1 was recaptured at rank 12 for D2, and D1 was recaptured at rank 27 for D3. The top10 average indication accuracy would be (1+0+0) / 3 == 33%, whereas the top25 would be (1+1+0) / 3 == 66%, and the top50 would be 100%. 

### Canbenchmark - classic
Below is how to run the benchmarking code. The first argument of `canbenchmark()` is the extension to put on the output files ("results_analysed_named", "raw_results", and "summary"). 

In [6]:
cando.canbenchmark('example')

	aia
top10	20.156
top25	26.906
top50	33.976
top100	43.034
top2162	100.000
top1%	25.155
top5%	44.260
top10%	55.990
top50%	87.256
top100%	100.000




Below is the printed summary file for the classic canbenchmark.

In [7]:
with open("summary-example.tsv", 'r') as f:
    for line in f:
        print(line.strip('\n'))

	top10	top25	top50	top100	top2162	top1%	top5%	top10%	top50%	top100%
aia	20.156	26.906	33.976	43.034	100.000	25.155	44.260	55.990	87.256	100.000
apa	35.109	47.776	59.571	71.366	100.000	44.854	72.526	82.681	97.166	100.000
ic	792	902	984	1084	1570	872	1098	1215	1484	1570



### Canbenchmark - associated
There are variations of the benchmarking code which may be of interest to some users. For example, not every compound in the mapping files are associated with a disease. This can decrease performance. We can have these compounds removed by running another benchmarking method called `canbenchmark_associated()`, which will automatically filter out these non-associated compounds. 

In [8]:
cando.canbenchmark_associated('example_associated')

Making CANDO copy with only benchmarking-associated compounds
Reading signatures from matrix...
Done reading signatures.

Computing rmsd distances...
Done computing rmsd distances.

	aia
top10	22.845
top25	31.250
top50	39.102
top100	49.247
top1403	100.000
top1%	25.161
top5%	43.697
top10%	54.863
top50%	86.114
top100%	100.000




Below is the printed summary file for the associated canbenchmark.

In [9]:
with open("summary-example_associated.tsv", 'r') as f:
    for line in f:
        print(line.strip('\n'))

	top10	top25	top50	top100	top1403	top1%	top5%	top10%	top50%	top100%
aia	22.845	31.250	39.102	49.247	100.000	25.161	43.697	54.863	86.114	100.000
apa	40.528	55.030	66.687	77.173	100.000	44.716	71.952	81.874	96.840	100.000
ic	830	957	1043	1150	1570	877	1090	1206	1478	1570



### Canbenchmark - continuous
Yet another variation of our benchmarking is `continuous=True`. This method identifies percentiles for the compound distances which define our cutoffs. Rather than top10 compounds, we can define an RMSD cutoffs that will not punish a compound that may fall at rank 11, but is still very similar to the hold-out compound. These RMSD cutoffs are determined empirically. 

In [10]:
cando.canbenchmark('example_continuous', continuous=True)

	aia
0.1%ile	10.476
.25%ile	13.654
0.5%ile	16.388
1%ile	19.809
5%ile	34.513
10%ile	43.077
20%ile	55.003
33%ile	66.618
50%ile	77.932
100%ile	100.000




Below is the printed summary file for the continuous canbenchmark.

In [11]:
with open('summary-example_continuous.tsv', 'r') as f:
    for line in f:
        print(line.strip('\n'))

	0.1%ile	.25%ile	0.5%ile	1%ile	5%ile	10%ile	20%ile	33%ile	50%ile	100%ile
aia	10.476	13.654	16.388	19.809	34.513	43.077	55.003	66.618	77.932	100.000
apa	15.535	21.518	26.916	33.225	53.550	62.135	72.223	80.316	87.515	100.000
ic	566	638	690	740	932	1030	1155	1279	1391	1570



## Canpredict

An important part of CANDO is generating putative drug candidates for a specific disease and predicting indications for which added or current drugs in our library can be therapetuic. We can do this with the canpredict functions.

### Canpredict - compounds
Generating putative drug candidates for a specific disease is one way we may want to use the predictive power of our platform. For this, we use the `canpredict_compounds()` function, which basically uses a consensus method to rank putative compounds based on how many times they show up as similar to drugs approved to treat the disease within some cutoff. The default cutoff is 10 (the most stringent from benchmarking), but this can be varied. In its current implementation, `canpredict_compounds()` ranks compounds based on the consensus count and uses it's average rank as the tie-breaker. 

Let's make predictions for human immunodeficiency virus (HIV). First, let's search our indication database for the appropriate MeSH ID. 

In [12]:
cando.search_indication('HIV')

Matches exactly containing HIV:
id             	name
MESH:D015658	HIV Infections
MESH:D006679	HIV Seropositivity
MESH:D019247	HIV Wasting Syndrome
MESH:D039682	HIV-Associated Lipodystrophy Syndrome

Matches using string distance:
id             	name
MESH:D015658	HIV Infections


The one we want is "HIV Infections" with an `id_` of "MESH:D015658". This is the input for `canpredict_compounds()`, but first let's see more about this indication. We 

In [13]:
hiv = cando.get_indication("MESH:D015658")
print('Number of drugs associated with {}: {}'.format(hiv.name, len(hiv.compounds)))
for cmpd in hiv.compounds:
    print('{}\t{}'.format(cmpd.id_, cmpd.name))

Number of drugs associated with HIV Infections: 32
128	nevirapine
570	lamivudine
599	pentamidine
369	zidovudine
855	hydroxyurea
579	adefovir_dipivoxil
513	stavudine
755	didanosine
918	atazanavir
567	delavirdine
896	abacavir
795	zalcitabine
670	ribavirin
398	foscarnet
377	ritonavir
1257	deferasirox
1570	raltegravir
564	amprenavir
1469	etravirine
1350	maraviroc
889	thalidomide
1123	fosamprenavir
114	indinavir
803	isoniazid
1071	saquinavir
1249	lopinavir
55	vitamin_a
110	nelfinavir
297	miglustat
489	efavirenz
219	methadone
1526	beta_carotene


There are 32 drugs associated with HIV Infections, which makes sense given how well-studied it is as a disease. Most seem to be protease inhibitors (saquinavir, fosamprenavir) or nucleoside analogs (zalcitabine, didanosine). 

We can now make predictions for other drugs/compounds that may be useful against HIV. This list is very long due to the large amount of drugs approved to treat HIV, so we should look at only the top10 (default) by setting `n=10`. We will also only print the first 10 compounds (`topX=10`).

In [14]:
cando.canpredict_compounds("MESH:D015658", n=10, topX=10)

32 compounds found for MESH:D015658 --> HIV Infections
Generating compound predictions using top10 most similar compounds...

rank	score1	score2	id	approved	name
1	5	5.8	736	true    	emtricitabine
2	4	2.0	1101	true    	darunavir
3	4	2.2	310	true    	trifluridine
4	4	3.0	1732	true    	cobicistat
5	4	3.5	209	true    	floxuridine
6	4	4.2	1660	true    	sofosbuvir
7	4	5.0	139	true    	idoxuridine
8	3	3.3	318	true    	gemcitabine
9	3	3.7	1409	true    	telaprevir
10	3	3.7	854	true    	ganciclovir




Perhaps using the top10 compounds is not encompassing enough. Let's change it to top25 (n=25) and see if the predictions change. We will still print the first 10 compounds.

In [15]:
cando.canpredict_compounds("MESH:D015658", n=25, topX=10)

32 compounds found for MESH:D015658 --> HIV Infections
Generating compound predictions using top25 most similar compounds...

rank	score1	score2	id	approved	name
1	7	14.7	319	true    	entecavir
2	7	18.9	10	true    	pyridoxal_phosphate
3	6	7.5	1732	true    	cobicistat
4	6	7.8	736	true    	emtricitabine
5	6	10.0	1660	true    	sofosbuvir
6	6	13.2	253	true    	cidofovir
7	6	17.5	1721	true    	tedizolid_phosphate
8	5	3.6	1101	true    	darunavir
9	5	7.8	646	true    	aciclovir
10	5	9.2	206	true    	piperacillin




We can show more results by changing the `topX` parameter. Let's print 25. 

We can also show the compounds that are already approved for 'HIV Infections' - indicated by an asterisk in the 'approved' column, by changing the `keep_associated` parameter to `True`. In this case, we recaptured associated therapies abacavir and amprenavir. 

<span style="color:red">(Note: this function is heavily reliant on the input indication mapping.)</span>

In [16]:
cando.canpredict_compounds("MESH:D015658", n=25, topX=20, 
                           keep_associated=True)

32 compounds found for MESH:D015658 --> HIV Infections
Generating compound predictions using top25 most similar compounds...

rank	score1	score2	id	approved	name
1	7	14.7	319	true    	entecavir
2	7	18.9	10	true    	pyridoxal_phosphate
3	6	7.5	1732	true    	cobicistat
4	6	7.8	736	true    	emtricitabine
5	6	10.0	1660	true    	sofosbuvir
6	6	13.2	253	true    	cidofovir
7	6	16.3	896	true*   	abacavir
8	6	17.5	1721	true    	tedizolid_phosphate
9	5	3.6	1101	true    	darunavir
10	5	7.8	646	true    	aciclovir
11	5	9.2	206	true    	piperacillin
12	5	11.2	854	true    	ganciclovir
13	5	11.4	1623	true    	boceprevir
14	5	12.4	524	true    	metaxalone
15	5	12.8	1792	true    	sonidegib
16	5	13.8	766	true    	repaglinide
17	5	17.4	946	true    	capecitabine
18	5	18.8	1555	true    	mangafodipir
19	4	2.2	310	true    	trifluridine
20	4	3.0	564	true*   	amprenavir




## Canpredict - de novo

Sometimes there are no drugs associated with an indication (whether it be in reality or in your input indication mapping); in those cases we can use the `canpredict_denovo()` function to suggest putative candidates. Basically, this function counts/sums the number of protein interaction scores above a set threshold for each compound and ranks them according to frequency and strength. This is particularly useful for pathogen proteomes (like SARS-CoV-2 or bacterial proteomes) or for finding which compounds target a subset of proteins of interest (e.g. kinases). In our case, we have a diverse set of proteins, which would render the results meaningless. Instead, we can input a list of protein IDs in which we are interested. We have precompiled a list of bacterial proteins already present in the sample matrix for convenience. Note: if we had read in an indication-genes mapping file, we could use the `ind_id=` parameter to automatically select all proteins associated with said indication. 

In [17]:
bacterial_proteins = ["3mk7C", "3eziA", "1u2mC", "3atsA", "2x4mA", 
                      "4wliA", "1t4aA", "4zxkA", "1zhhA", "1eb0A", 
                      "4nqwB", "2gqrA", "2qlcA", "3rf1A", "2xpwA"]

cando.canpredict_denovo(method='count', threshold=0.6, topX=45, proteins=bacterial_proteins)

Printing the 45 highest predicted compounds...

rank	score1	score2	id	approved	name
1	5	4.438	25	true    	adenosine_monophosphate
2	4	4.0	1920	true    	magnesium_chloride
3	4	4.0	1162	true    	magnesium
4	4	3.034	504	true    	adenosine
5	4	3.034	85	true    	vidarabine
6	4	2.737	50	true    	nadh
7	2	2.0	2006	true    	sodium_sulfide
8	2	2.0	1982	true    	sodium_iodide
9	2	2.0	1801	true    	sodium_chloride
10	1	1.0	2021	true    	ferrous_bisglycinate
11	1	1.0	2018	true    	magnesium_glycinate
12	1	1.0	1959	true    	choline_c-11
13	1	1.0	1950	true    	sodium_ferric_gluconate_complex
14	1	1.0	1862	true    	choline_c_11
15	1	1.0	1795	true    	iron_saccharate
16	1	1.0	1757	true    	chlortetracycline
17	1	1.0	1566	true    	polidocanol
18	1	1.0	1301	true    	sucrose
19	1	1.0	1242	true    	zinc
20	1	1.0	1160	true    	calcium
21	1	1.0	1074	true    	levodopa
22	1	1.0	1013	true    	kanamycin
23	1	1.0	927	true    	streptomycin
24	1	1.0	665	true    	pentoxifylline
25	1	1.0	49	true    	l-threonine
26	1

Interestingly, multiple antibiotics receive scores higher than the cutoff of 0.6, including **kanamycin, streptomycin, netilmicin, doxycycline**, among others. 

### Canpredict - indications


Below we print the first 10 indications predicted for Paromomycin using the top10 most similar compounds. Again, this tallies how many times specific diseases show up as associated with the top10 most similar compounds to paromomycin. 

In [18]:
cando.canpredict_indications(cando.compounds[1191], 
                             n=10, topX=10)

Using CANDO compound paromomycin
Compound has id 1191 and index 1191
Comparing signature to all CANDO compound signatures...
Generating indication predictions using top10 most similar compounds...
Printing the 10 highest predicted indications...

rank	score	ind_id    	indication
1	4	MESH:D004927	Escherichia coli Infections
2	4	MESH:D008581	Meningitis
3	3	MESH:D004697	Endocarditis, Bacterial
4	3	MESH:D007710	Klebsiella Infections
5	3	MESH:D011014	Pneumonia
6	3	MESH:D018805	Sepsis
7	3	MESH:D013203	Staphylococcal Infections
8	3	MESH:D014376	Tuberculosis
9	3	MESH:D014552	Urinary Tract Infections
10	3	MESH:D011512	Proteus Infections



Not surprisingly, all of the predictions are infections of some sort (aka: most antibiotics share a great deal of structural similarity with each other). 

### Similar compounds
`similar_compounds()` prints the first `n` most similar compounds for a given compound. This, like `canpredict_indications()` can be used with cando compounds, `cando_cmpd`, or novel compounds with a signature file (we will explore this later).

Below we print the first 10 most similar compounds to Paromomycin.

In [19]:
cando.similar_compounds(cando.compounds[1191], n=10)

Using CANDO compound paromomycin
Compound has id 1191 and index 1191
Comparing signature to all CANDO compound signatures...
Printing top10 most similar compounds...

rank	dist	id	name
1	0.010	328	framycetin
2	0.018	1315	ribostamycin
3	0.036	353	amikacin
4	0.039	547	tobramycin
5	0.046	2130	esculin
6	0.052	1013	kanamycin
7	0.052	2036	polydatin
8	0.052	1492	arbekacin
9	0.056	2125	calcium_glubionate
10	0.060	927	streptomycin
11	0.060	173	acarbose




As expected, most compounds are known antibiotics. 

## Machine learning with CANDO
The "proteomic vectors" within CANDO lend themselves well to machine learning to perhaps learn more complex relationships between the proteins within the vector and their impacts on the treatment of diseases. CANDO has built-in ML algorithms that allow for two main functionalities:
1. Benchmark the platform using a hold-one-out protocol very similar to canbenchmark
2. Make predictions for novel or non-associated compounds that may be therapeutic for a given disease

The ML module currently supports 2 algorithms: random forests and logistic regression. The models are trained on drugs approved for the disease (positive classes) and an equal number of randomly selected "neutral samples", which are drugs/compounds not approved for the disease (negative samples). Random seeds may be set to ensure the same compounds are used in training. 

### ML - benchmark
We have the option to benchmark the platform with an ML algorithm - this module outputs files very similar to canbenchmark. For this tutorial, we will skip this function as it requires a great deal of time to complete (training a separate model for EVERY drug-disease association, basically). The command to do so with a logistic regression classifier is below, feel free to run it! The `'out='` flag defines the name of the output files. Again, only diseases with 2+ compounds associated are benchmarked. 

`cnd.ml(method='rf', benchmark=True, seed=50, out='test_rf')`

### ML - predict
We can also use this module to predict if a certain compound may be therapeutic for a given disease. We can use the 
`'predict='` flag to specify a list of compounds that we wish to predict with the classifier. Let's use three drugs, imatinib, buprenorphine, and lisdexamfetamine, and see if they are predicted to have antibiotic activity using a random forest classifier. 

In [20]:
baci = cando.get_indication('MESH:D001424')

imat = cando.get_compound(483)
bup = cando.get_compound(775)
lamf = cando.get_compound(1094)

cando.ml(method='rf', effect=baci, benchmark=False, seed=50, predict=[imat, bup, lamf])

Indication: Bacterial Infections
Leave-one-out cross validation: TP=36, FP=55, FN=19, TN=0, Acc=32.727
	Compound	Prob
	imatinib	0.190
	buprenorphine	0.640
	lisdexamfetamine	0.130


## Custom protein subsets and signatures
It may be useful for some users to probe compound-protein interaction similarity, but only in the context of a few particular proteins (e.g. set of kinase inhibitors). Instead of generating a new matrix with all of these proteins and their corresponding interaction values, which can begin to take up a lot of storage if done multiple times, the `'protein_set='` flag can be specified during the instantiation of the CANDO object. This flag contains the path to the protein subset the user wishes to use, which is simply a list of UniProt protein IDs. The CANDO object will automatically check for each ID if it either simply matches any UniProt IDs within the matrix or if that UniProt ID is associated with any PDB chains within the matrix (based on a mapping from the SIFTs project). If there are matches, the CANDO object will now contain Compound objects with only those protein interaction values in their signatures. Below is how to use this functionality - this time we can search for the bacterial proteins from the previous `canpredict_denovo()` section, but filter the proteins from the start using a list of UniProt IDs. 

In [21]:
cando_subset = cnd.CANDO(cmpd_map, ind_map, matrix=matrix_file, 
                         compute_distance=True, protein_set=protein_set,
                         dist_metric=dist_metric, ncpus=ncpus)

print("Number of proteins in new signature =", len(cando_subset.proteins))

Reading signatures from matrix...
Editing signatures according to proteins in example-bac-prots.txt...
15 proteins in example-bac-prots.txt mapped to 15 proteins in example-matrix.tsv.
Done reading signatures.

Computing rmsd distances...
Done computing rmsd distances.

Number of proteins in new signature = 15


The signature was successfully edited to 15 proteins. Note: this does not nececessarily mean the each UniProt ID had a corresponding PDB match -- multiple PDB chains can be associated with a given UniProt ID. 

We can also repeat all benchmarks and predictive algorithms with the new signatures. Below is the default benchmarking results with the new signatures. 

In [22]:
cando_subset.canbenchmark('test_subset')

	aia
top10	17.070
top25	24.055
top50	30.694
top100	40.226
top2162	100.000
top1%	22.414
top5%	41.363
top10%	53.126
top50%	87.278
top100%	100.000




Let's repeat the ML code from above, but this time let's use logistic regression. 

In [23]:
baci = cando_subset.get_indication('MESH:D001424')

imat = cando_subset.get_compound(483)
bup = cando_subset.get_compound(775)
lamf = cando_subset.get_compound(1094)

cando_subset.ml(method='log', effect=baci, benchmark=False, 
                seed=50, predict=[imat, bup, lamf])

Indication: Bacterial Infections
Leave-one-out cross validation: TP=37, FP=55, FN=18, TN=0, Acc=33.636
	Compound	Prob
	imatinib	0.514
	buprenorphine	0.533
	lisdexamfetamine	0.400


As you can see, the output probabilities significantly change for the three test compounds (imatinib: 0.190->0.514,
	buprenorphine: 0.640->0.533, lisdexamfetamine: 0.130->0.400), as well as a slight change in performance (Acc: 32.7->33.6), illustrating the impact of protein subset composition on model training. 

## Generate proteomic signatures for new compounds 

The CANDO platform contains an extensive library of approved drugs (2,162) and other compounds (additional 6,590) from DrugBank. However, if you wish to predict indications or similar drugs for a compound that is not present in our library, we make it possible with the following series of functions.

First, you must have the compound properly formatted in PDB file format. There are many programs that provide conversion among many chemical file formats if you require, such as OpenBabel.

Next, you can run the `generate_scores()` function. This will populate a tsv file with tanimoto similarity scores (jaccard index) of the provided compound to all binding site ligands in our database. These values will be used for the generation of the drug-proteome signature. The fingerprint used for tanimoto score will be defined by the input for variable 'fp'. The default for this is "rd_ecfp4". The tsv file will be saved with the name you provide in cmpd_id (e.g. cmpd_id=7561; "7561.tsv") and it will be saved in the directory provided as the out_path along with the fingerprint defined (e.g. out_path="examples"; "examples/rd_ecfp4/7561.tsv"). Any novel compound structure file should be named in the format `####.pdb` such as 1000.pdb or 25.pdb. The output of `generate_scores()` is within the directory named for the fingerprint method used (e.g. "rd_ecfp4/") and is named in the format `####_scores.tsv` (e.g. 1000_scores.tsv). 

Next, `generate_signature()` is called which creates a file with the interaction scores against all proteins in the `prot_scores` input file. It is saved in a format similar to `generate_scores()` (e.g. 1000_signature.tsv). NOTE: the interaction score protocol must match the input matrix protocol, which was 'C' as above. If we used a different protocol, these scores would not be directly comparable. 

In [24]:
cnd.generate_scores(fp="rd_ecfp4", cmpd_pdb="8100.pdb", 
                    out_path=".")
cnd.generate_signature(cmpd_scores="rd_ecfp4/8100_scores.tsv",
                       prot_scores=prot_scores, interaction_score='C')

/Users/williammangione/Documents/CANDO/cando/v2.0/ligands_fps/rd_ecfp4.tsv [164.5 MB] [] [Time:  0:06:47] 


Ligand fingerprints downloaded.
Generating rd_ecfp4 fingerprints and scores...
Calculating tanimoto scores for compound 8100 against all binding site ligands...
Tanimoto scores written to ./rd_ecfp4/8100_scores.tsv

Compiling compound scores...
Compiling binding site scores...
Generating interaction signature...
8100
Signature written to 8100_signature.tsv.
Signature generation took 0 s to finish.


We then must add the compound to the existing `CANDO` object with the `add_cmpd()` function. The inputs are the newly generated signature file and the desired name (which below is "scy-635"). 

In [25]:
cando.add_cmpd("8100_signature.tsv", 'scy-635')

New compound is scy-635
New compound has id 2162 and index 2162


Now that our new compound is added to the platform, we can see what other compounds to which it is similar. We can use the `similar_compounds()` function from before to print those compounds. 

In [27]:
scy635 = cando.get_compound(2162)
cando.similar_compounds(scy635, n=10)

Using CANDO compound scy-635
Compound has id 2162 and index 2162
Comparing signature to all CANDO compound signatures...
Printing top10 most similar compounds...

rank	dist	id	name
1	0.020	6	cyclosporine
2	0.021	2154	glecaprevir
3	0.022	2064	asunaprevir
4	0.023	1741	edoxaban
5	0.023	2102	voxilaprevir
6	0.023	1454	udenafil
7	0.023	1129	cefazolin
8	0.024	1460	simeprevir
9	0.024	1712	ledipasvir
10	0.024	1132	cefotetan
11	0.024	1864	fimasartan




Given that scy-635 is a cyclosporine analog, this is an expected and good result as cyclosporine shows up at rank 1.

Finally, we can predict potential indications for which our new compound may be useful. Since cyclosporine is a known immunosuppressant, we should expect to find autoimmune or inflammatory indications. We can use the `canpredict_indications()` as before to print those results. 

In [29]:
cando.canpredict_indications(scy635, n=10, topX=10)

Using CANDO compound scy-635
Compound has id 2162 and index 2162
Comparing signature to all CANDO compound signatures...
Generating indication predictions using top10 most similar compounds...
Printing the 10 highest predicted indications...

rank	score	ind_id    	indication
1	3	MESH:D006526	Hepatitis C
2	2	MESH:D010019	Osteomyelitis
3	2	MESH:D019698	Hepatitis C, Chronic
4	1	MESH:D058186	Acute Kidney Injury
5	1	MESH:D000741	Anemia, Aplastic
6	1	MESH:D000744	Anemia, Hemolytic, Autoimmune
7	1	MESH:D001169	Arthritis, Experimental
8	1	MESH:D001172	Arthritis, Rheumatoid
9	1	MESH:D001327	Autoimmune Diseases
10	1	MESH:D001528	Behcet Syndrome



Interestingly, hepatitis C is the strongest hit, which is a good result given it is [currently being investigated as a potential treatment for Hep C](https://www.drugbank.ca/drugs/DB12451). 

## Virtual screening in CANDO
Though the CANDO platform is mainly intended for multitarget drug discovery and repurposing, there still exists the option to check the top compound hits for a given protein. This is accomplished via the `virtual_screen()` function. Let's test the top hits for two of the known bacterial proteins from above, namely "1u2mC", "3atsA". 

In [32]:
cando.virtual_screen("1u2mC")
cando.virtual_screen("3atsA")

Protein is 1u2mC
rank	score	id	approved	name
1	0.759	25	true    	adenosine_monophosphate
2	0.676	50	true    	nadh
3	0.649	306	true    	streptozocin
4	0.618	504	true    	adenosine
5	0.618	85	true    	vidarabine
6	0.548	1306	true    	flavin_adenine_dinucleotide
7	0.525	1602	true    	hyaluronic_acid
8	0.517	1336	true    	gluconolactone
9	0.5	1334	true    	lactose
10	0.493	13	true    	ademetionine

Protein is 3atsA
rank	score	id	approved	name
1	1.0	2006	true    	sodium_sulfide
2	1.0	1982	true    	sodium_iodide
3	1.0	1959	true    	choline_c-11
4	1.0	1920	true    	magnesium_chloride
5	1.0	1862	true    	choline_c_11
6	1.0	1801	true    	sodium_chloride
7	1.0	1162	true    	magnesium
8	1.0	1013	true    	kanamycin
9	1.0	927	true    	streptomycin
10	1.0	504	true    	adenosine



Streptozocin is the 3rd ranked hit for 1u2mC, which suggests it shares significant structural similarity to a ligand known/predicted to bind to this protein. Similarly, both kanamycin and streptomycin are tied for the top rank with a perfect score of 1.0 for 3atsA, which means both of those drugs are known or strongly predicted to bind to this protein. 